In [1]:
import torch
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile
import pandas as pd
from os.path import join as jp
import os
import matplotlib.pyplot as plt 
import seaborn as sns 
import matplotlib

/Users/davidamat/anaconda3/envs/vault_pytorch_lightgcn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import numpy as np
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
from torch import nn, optim, Tensor

from torch_geometric.utils import structured_negative_sampling
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn import LGConv
import scipy.sparse as sp

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [137]:
seed = 42
torch.manual_seed(seed)

# Downloading the data

- From Kaggle: https://www.kaggle.com/datasets/somnambwl/bookcrossing-dataset/
- Destination path: /Users/davidamat/Documents/david/learning/graph/data

In [5]:
path_data = "/Users/davidamat/Documents/david/learning/graph/data/book-crossing/"

In [6]:
path_ratings = jp(path_data, 'Ratings.csv')
path_users = jp(path_data, 'Users.csv')
path_books = jp(path_data, 'Books.csv')


ratings = pd.read_csv(path_ratings, sep=';', encoding='latin-1')
users = pd.read_csv(path_users, sep=';', encoding='latin-1')
books = pd.read_csv(path_books, sep=';', encoding='latin-1', on_bad_lines="skip")

# Preprocessing the Book-Crossing dataset


In [8]:
# Identifiers
books_ids = books['ISBN'].unique()
user_ids = users['User-ID'].unique()

# Ratings as df
df = ratings.copy()

# Mask only ratings of books and users that appear on the master tables of each one
mask_books_ids = df['ISBN'].isin(books_ids)
mask_users_ids = df['User-ID'].isin(user_ids)
df = df.loc[mask_books_ids & mask_users_ids]

# Keep the 100k highest ratings
df = df[df['Rating'] >= 8].iloc[:100000]

In [9]:
# Create mappings
user_mapping = {userid: i for i, userid in enumerate(df['User-ID'].unique())}
item_mapping = {isbn: i for i, isbn in enumerate(df['ISBN'].unique())}

# Count users and items
num_users = len(user_mapping)
num_items = len(item_mapping)
num_total = num_users + num_items

In [10]:
# Construct the IDS columns
df_ids = df.copy()
df_ids["u_id"] = df_ids["User-ID"].map(user_mapping)
df_ids["b_id"] = df_ids["ISBN"].map(item_mapping)

In [11]:
print("Users", num_users)
print("Items", num_items)
print("Total Users and Items", num_total)

Users 19557
Items 56913
Total Users and Items 76470


In [12]:
df

,User-ID,ISBN,Rating
9586,12,1879384493,10
9591,16,0345402871,9
9607,26,0446310786,10
9608,26,0449005615,9
9609,32,0060168013,8
...,...,...,...
480298,114601,0743419049,10
480303,114604,0618002219,9
480314,114611,0445409134,8
480315,114611,0446313033,8


## Edge Indices

In [13]:
# Build the adjacency matrix based on user ratings:

# 1) Take the column of users and convert their ID into the internal ID
user_ids = torch.LongTensor([user_mapping[i] for i in df['User-ID']])

# 2) Take the column of items and convert their ID into the internal ID
item_ids = torch.LongTensor([item_mapping[i] for i in df['ISBN']])

# 3) Create the edge tensor as the relationship between 1) and 2) (they come from ratings matrix)
edge_index = torch.stack((user_ids, item_ids))

In [16]:
user_ids

tensor([    0,     1,     2,  ..., 19556, 19556, 19556])

In [14]:
edge_index

tensor([[    0,     1,     2,  ..., 19556, 19556, 19556],
        [    0,     1,     2,  ..., 56910, 56911, 56912]])

# Splitting dataset

In [17]:
# Create training, validation, and test adjacency matrices
train_index, test_index = train_test_split(range(len(df)), test_size=0.2, random_state=0)
val_index, test_index = train_test_split(test_index, test_size=0.5, random_state=0)

In [18]:
print("Train:", sorted(train_index)[:20])
print("Test:", sorted(test_index)[:20])
print("Valid:", sorted(val_index)[:20])

Train: [0, 1, 2, 4, 5, 7, 8, 10, 11, 12, 13, 15, 16, 18, 19, 20, 21, 22, 24, 27]
Test: [3, 14, 17, 23, 34, 48, 52, 56, 60, 63, 65, 68, 79, 109, 110, 117, 157, 165, 187, 196]
Valid: [6, 9, 25, 26, 36, 41, 51, 54, 69, 72, 90, 105, 119, 121, 125, 128, 133, 151, 156, 166]


In [19]:
# Slice train/test/valid
train_edge_index = edge_index[:, train_index]
val_edge_index = edge_index[:, val_index]
test_edge_index = edge_index[:, test_index]

In [20]:
# Interaction values (assume all 1s)
train_edge_values = torch.ones_like(train_edge_index[0,:])

In [23]:
train_edge_index.shape

torch.Size([2, 80000])

In [139]:
train_edge_values.shape

torch.Size([80000])

## Adjacency Matrix

In [33]:
# Interaction matrix
R = sp.coo_matrix((train_edge_values, (train_edge_index[0], train_edge_index[1])), shape=(num_users, num_items))
R = R.tolil()

In [39]:
# Adjacency matrix
MN = num_users + num_items
adj_mat = sp.dok_matrix((MN, MN), dtype=np.float32)
adj_mat = adj_mat.tolil()

In [40]:
adj_mat

<76470x76470 sparse matrix of type '<class 'numpy.float32'>'
	with 0 stored elements in List of Lists format>

In [41]:
%%time
# Fill adjacency matrix
adj_mat[:num_users, num_users:] = R
adj_mat[num_users:, :num_users] = R.T

CPU times: user 1min 26s, sys: 9.05 s, total: 1min 36s
Wall time: 1min 37s


In [44]:
# Rowsum
rowsum = np.array(adj_mat.sum(1))
degrees = rowsum.ravel()

In [52]:
#dd = pd.Series(degrees).value_counts()
# dd[:15].plot(kind='bar', title="Degree distrib", xlabel="Degree of node")

In [53]:
# Inverse of the Degree matrix
d_inv = np.power(rowsum, -0.5).flatten()
d_inv[np.isinf(d_inv)] = 0.
d_mat = sp.diags(d_inv)

In [54]:
# Normalized Adjacency Matrix
norm_adj = d_mat.dot(adj_mat)
norm_adj = norm_adj.dot(d_mat)
# norm_adj = norm_adj.tocsr()

In [55]:
norm_adj

<76470x76470 sparse matrix of type '<class 'numpy.float32'>'
	with 160000 stored elements in Compressed Sparse Row format>

## LightGCN

In [140]:
class LightGCN(nn.Module):
    def __init__(self, num_users, num_items, edge_index, edge_values, num_layers=4, dim_h=64):
        super().__init__()
        
        self.num_users = num_users
        self.num_items = num_items
        self.num_layers = num_layers
        self.emb_users = nn.Embedding(num_embeddings=self.num_users, embedding_dim=dim_h)
        self.emb_items = nn.Embedding(num_embeddings=self.num_items, embedding_dim=dim_h)
        self.edge_index = edge_index
        self.edge_values = edge_values
        self.adj_mat = self.compute_norm_adj_matrix(edge_index, edge_values)
        self.sp_adj_mat = self._convert_sp_mat_to_sp_tensor(self.adj_mat)
        self.alpha = 1/(self.num_layers+1)

        # self.convs = nn.ModuleList(LGConv() for _ in range(num_layers))

        nn.init.normal_(self.emb_users.weight, std=0.01)
        nn.init.normal_(self.emb_items.weight, std=0.01)
        
    @staticmethod
    def _convert_sp_mat_to_sp_tensor(X):
        coo = X.tocoo().astype(np.float32)
        row = torch.Tensor(coo.row).long()
        col = torch.Tensor(coo.col).long()
        index = torch.stack([row, col])
        data = torch.FloatTensor(coo.data)
        return torch.sparse.FloatTensor(index, data, torch.Size(coo.shape))
    
    def compute_norm_adj_matrix(self, edge_index, edge_values):
        num_users = self.num_users
        num_items = self.num_items
        # Interaction matrix
        R = sp.coo_matrix((
            edge_values, 
            (edge_index[0], edge_index[1])),
            shape=(num_users, num_items))
        R = R.tolil()
        
        # Adjacency matrix
        MN = self.num_users + self.num_items
        adj_mat = sp.dok_matrix((MN, MN), dtype=np.float32)
        adj_mat = adj_mat.tolil()
        
        # Fill adjacency matrix
        adj_mat[:num_users, num_users:] = R
        adj_mat[num_users:, :num_users] = R.T
        
        # Degrees
        rowsum = np.array(adj_mat.sum(1))
        
        # Inverse of the Degree matrix
        d_inv = np.power(rowsum, -0.5).flatten()
        d_inv[np.isinf(d_inv)] = 0.
        d_mat = sp.diags(d_inv)
        
        # Normalized Adjacency Matrix
        norm_adj = d_mat.dot(adj_mat)
        norm_adj = norm_adj.dot(d_mat)
        return norm_adj
        

    def forward(self):
        # Keep track of starting embeddings for feeding into the BPR Loss 
        # for regularizing the learned embedding params
        emb0_users = self.emb_users.weight
        emb0_items = self.emb_items.weight
                
        # Embedding is dimension M + N
        emb = torch.cat([emb0_users, emb0_items])
        embs = [emb]

        # For each layer
        for layer_i in range(self.num_layers):
            emb = torch.sparse.mm(self.sp_adj_mat, emb)
            embs.append(emb)

        emb_final = self.alpha * torch.mean(torch.stack(embs, dim=1), dim=1)

        embf_users, embf_items = torch.split(emb_final, [self.num_users, self.num_items])

        return embf_users, emb0_users, embf_items, emb0_items

In [ ]:
model = LightGCN(
    num_users=nu, 
    num_items=ni, 
    edge_index=toy_edge_index,
    edge_values=toy_edge_values,
    num_layers=1, 
    dim_h=embdi
)

In [141]:
seed = 42
torch.manual_seed(seed)
toy_edge_index = torch.tensor(
    [[0, 0, 1, 1, 2, 2, 3, 3, 4, 4],
     [0, 1, 0, 2, 1, 2, 1, 2, 0, 2]]
)

structured_negative_sampling(toy_edge_index)

(tensor([0, 0, 1, 1, 2, 2, 3, 3, 4, 4]),
 tensor([0, 1, 0, 2, 1, 2, 1, 2, 0, 2]),
 tensor([2, 2, 1, 4, 3, 0, 0, 4, 4, 3]))

In [152]:
import torch
from torch_geometric.utils import structured_negative_sampling

# Define the toy edge index
toy_edge_index = torch.tensor(
    [[0, 0, 1, 1, 2, 2, 3, 3, 4, 4],
     [0, 1, 0, 2, 1, 2, 1, 2, 0, 2]]
)

# Assume you have 5 users and 3 items
num_users = 5
num_items = 3


# Use structured_negative_sampling
neg_edge_index = structured_negative_sampling(toy_edge_index, num_nodes=3)

# Print the results
print("Original Edge Index:")
print(toy_edge_index)
print("\nNegative Edge Index:")
print(neg_edge_index)

Original Edge Index:
tensor([[0, 0, 1, 1, 2, 2, 3, 3, 4, 4],
        [0, 1, 0, 2, 1, 2, 1, 2, 0, 2]])

Negative Edge Index:
(tensor([0, 0, 1, 1, 2, 2, 3, 3, 4, 4]), tensor([0, 1, 0, 2, 1, 2, 1, 2, 0, 2]), tensor([2, 2, 1, 1, 0, 0, 0, 0, 1, 1]))


In [153]:
neg_edge_index

(tensor([0, 0, 1, 1, 2, 2, 3, 3, 4, 4]),
 tensor([0, 1, 0, 2, 1, 2, 1, 2, 0, 2]),
 tensor([2, 2, 1, 1, 0, 0, 0, 0, 1, 1]))

In [144]:
def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))

In [149]:
num_nodes = maybe_num_nodes(toy_edge_index, (num_users, num_items))

In [146]:
row, col = edge_index.cpu()

In [147]:
row

tensor([0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4])

In [148]:
col

tensor([0, 1, 2, 0, 2, 1, 2, 1, 2, 0, 2])

In [151]:
row.size(0)

11

In [150]:
rand = torch.randint(num_nodes, (row.size(0), ), dtype=torch.long)

TypeError: randint() received an invalid combination of arguments - got (tuple, tuple, dtype=torch.dtype), but expected one of:
 * (int high, tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int high, tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int low, int high, tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int low, int high, tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
